<a href="https://colab.research.google.com/github/MarcoParola/structural_health_monitoring/blob/main/utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Utility notebook**

Here I declare some functions and global variables usefull in all other notebooks

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sn
from scipy import stats
from tensorflow.keras import optimizers, regularizers
import keras
from keras import layers
from sklearn import metrics 

In [ ]:
BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/thesis2/'

DATA_PATH = BASE_PATH + 'data/'
MODELS_PATH = BASE_PATH + 'models/'

DATASET = DATA_PATH + 'dataset.npy'

NUMBER_OF_SENSORS = 16
ENCODED_LENGTH = 70
PATIENCE = 15
MAX_EPOCHS = 10

earlyStopCallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                     mode='min', verbose=1, 
                                                     patience=PATIENCE, 
                                                     restore_best_weights=True)

# Load ad preprocess data

Utility function to load a dataset

In [ ]:
def load_dataset(dataset):
  with open(DATA_PATH + dataset + '.npy', 'rb') as f:
    dataset = np.load(f, allow_pickle=True)

  print(dataset.shape)
  return dataset

Utility function to monitor the performance during the training process

In [ ]:
def plot_accurancy_loss(hist):
  plt.rcParams["figure.figsize"] = (11,6.5)
  font = {'size'   : 12}
  plt.rc('font', **font)

  acc_1 = hist.history['accuracy']
  val_acc_1 = hist.history['val_accuracy']
  loss_1 = hist.history['loss']
  val_loss_1 = hist.history['val_loss']
  
  epochs = range(len(acc_1))

  plt.figure()
  plt.plot(epochs, loss_1, 'C1', alpha=0.6, label='Training loss')
  plt.plot(epochs, val_loss_1, 'C2', alpha=0.6, label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend(prop={'size': 17})
  plt.show()

  plt.ylim(0,1)
  plt.plot(epochs, acc_1, 'C1', alpha=0.6, linewidth=2, label='Training acc')
  plt.plot(epochs, val_acc_1, 'C2', alpha=0.6, linewidth=2, label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend(prop={'size': 17})

  plt.show()

# **Function to evaluate model**

## Regressor

In [ ]:
def evaluateRegressor(model, testset, actual, minimal=False):
  pred = model.predict(testset)
  resid = np.squeeze(levels_test) - np.squeeze(pred)

  font = {'size'   : 13}
  plt.rc('font', **font)
  plt.rcParams["figure.figsize"] = (10,8)

  print('R^2: ', r2_score(levels_test, pred), '\nMAE', mean_absolute_error(levels_test, pred))

  if minimal:
    plt.plot([0.05, 0.25], [0.05, 0.25], c='orange', linewidth=4, alpha=.7)
    plt.scatter(np.squeeze(levels_test), np.squeeze(pred), s=5, alpha=.5)
    plt.title('Actual Damage Level vs Predicted values')
    plt.xlabel('Actual Damage Level')
    plt.ylabel('Predicted values')
    plt.show()

    x = range(len(resid))
    plt.title('Residuals')
    plt.xlabel('index')
    plt.ylabel('displacement')
    plt.plot([0, len(resid)], [0, 0], linewidth=4, c='orange', alpha=.8)
    plt.scatter(x, resid, s=5, alpha=.5)
    plt.show()

  

## Classifier

In [ ]:
def evaluateClassifier(model, testset, actual, minimal=False):
  pred = model.predict(testset)

  y_classes = pred.argmax(axis=-1)
  y_classes = to_categorical(y_classes)

  y_test_non_category = [ np.argmax(t) for t in labels_test ]
  y_predict_non_category = [ np.argmax(t) for t in y_classes ]
  conf_mat = confusion_matrix(y_test_non_category, y_predict_non_category)
  print(metrics.classification_report(y_test_non_category, y_predict_non_category))

  if minimal:
    # plot confusion matrix
    group_counts = ['{0:0.0f}'.format(value) for value in conf_mat.flatten()]
    group_percentages = ['{0:.2%}'.format(value) for value in (conf_mat / np.sum(conf_mat, axis=1)).flatten()]# conf_mat.flatten()/np.sum(conf_mat)]
    labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(9,9)
    fig, ax = plt.subplots(figsize=(14,10))
    plt.title('Confusion matrix')
    sn.heatmap(conf_mat, annot=labels, fmt='', cmap="OrRd", ax=ax)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
  

 ## Autoencoder

function to measure *mse* of an AE

In [ ]:
def evaluate_autoencoder(model, testSet):
  decoded_signals = model.predict(testSet)
  mse = (np.square(testSet - decoded_signals)).mean(axis=None)
  print('Mean square error: ', mse)

function to measure *mse* of a denoising_AE

In [ ]:
def evaluate_denoising_autoencoder(model, signals, noisy_signals):
  decoded_signals = model.predict(noisy_signals)
  mse = (np.square(signals - decoded_signals)).mean(axis=None)
  print('Mean square error: ', mse)